# Project: Battle of the Neighborhoods

This notebook is for the capstone project in the IBM Applied Data Science specialisation 

### Week 1: Setting up the Jupyter notebook

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 1st Assignment - Setup the Jupyter notebook on a hosted service
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Week 3: Segmenting and Clustering Neighborhoods in Toronto

#### Part 1: Storing Toronto Neighborhood Information into pandas DataFrame

In [8]:
import requests as req
from bs4 import BeautifulSoup as Soup

print("Imports done")

Imports done


In [22]:
# Fetch data from the resource link and store as a csv file
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = req.get(url).text
soup = Soup(res, 'html.parser')

# Use # for seperator since , can be used to group addresses
csv_data = soup.table.get_text().replace('\n\n\n', '\n').replace('\n\n', '#')[1:]  # Ignore the first seperator

# Write data into .csv file
with open("toronto.csv", "w") as f:
    f.write(csv_data)

In [33]:
# Read data into df
df = pd.read_csv("toronto.csv", header=0, sep='#')

# Remove any "Not Assigned" boroughs
df = df[df["Borough"] != "Not assigned"].reset_index(drop=True)

# Assign nbhood to have same name as borough if nbhood is "Not Assigned"
df["Neighborhood"] = df.T.apply(lambda x: x["Neighborhood"] if x["Neighborhood"] != "Not Assigned" else x["Borough"])

df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [34]:
df.shape

(103, 3)